# IR Lab WiSe 2023: Lemmatization

This tutorial shows how to configure and use custom lemmatizer in PyTerrier.

**Attention:** The scenario below is cherry-picked to explain the concept of stopword lists with a minimal example.


In [12]:
!wget https://files.webis.de/software/pyterrier-plugins/custom-terrier-token-processing-1.0-SNAPSHOT-jar-with-dependencies.jar -O /root/.pyterrier/custom-terrier-token-processing-0.0.1.jar

--2023-10-22 21:08:58--  https://files.webis.de/software/pyterrier-plugins/custom-terrier-token-processing-1.0-SNAPSHOT-jar-with-dependencies.jar
Resolving files.webis.de (files.webis.de)... 141.54.132.200
Connecting to files.webis.de (files.webis.de)|141.54.132.200|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 499865236 (477M) [application/java-archive]
Saving to: ‘/root/.pyterrier/custom-terrier-token-processing-0.0.1.jar’

/root/.pyterrier/cu 100%[===================>] 476.71M  24.7MB/s    in 21s     

2023-10-22 21:09:19 (22.8 MB/s) - ‘/root/.pyterrier/custom-terrier-token-processing-0.0.1.jar’ saved [499865236/499865236]



In [2]:
import pyterrier as pt
import pandas as pd
pd.set_option('display.max_colwidth', 0)

if not pt.started():
    pt.init(boot_packages=['mam10eks:custom-terrier-token-processing:0.0.1'])
    from jnius import autoclass

In [3]:
def lemmatize(t):
    lemmatizer = autoclass("org.terrier.terms.StanfordLemmatizer")()
    return lemmatizer.stem(t)

print('are =>', lemmatize('are'))
print('producer =>', lemmatize('producer'))
print('produces =>', lemmatize('produces'))
print('corpora =>', lemmatize('corpus'))

are => be
producer => producer
produces => produce
corpora => corpus


In [4]:
def stem(t):
    stemmer = autoclass("org.terrier.terms.PorterStemmer")()
    return stemmer.stem(t)

print('are =>', stem('are'))
print('producer =>', stem('producer'))
print('produces =>', stem('produces'))
print('corpora =>', stem('corpus'))

are => ar
producer => produc
produces => produc
corpora => corpu


In [14]:
documents = [
    {'docno': 'd1', 'text': 'A corpus is a dataset consisting of language resources.'},
    {'docno': 'd2', 'text': 'A corpus may contain documents in a single language or multiple languages.'},
    {'docno': 'd3', 'text': 'A semantic treebank is a collection of natural language sentences annotated with a meaning representation.'},
    {'docno': 'd4', 'text': 'A parallel text places two translation alongside each other which is often used to train machine learning translation.'},
]

topics = pd.DataFrame([
    {'qid': '1', 'query': 'text corpora'},
])

qrels = pd.DataFrame([
    {'qid': '1', 'docno': 'd1', 'relevance': 1},
    {'qid': '1', 'docno': 'd2', 'relevance': 1},
])

In [15]:
indexer = pt.IterDictIndexer("/tmp/index", overwrite=True, stemmer='PorterStemmer')
index_ref = indexer.index(documents)
index = pt.IndexFactory.of(index_ref)

bm25 = pt.BatchRetrieve(index, wmodel="BM25")

pt.Experiment([bm25], topics, qrels, eval_metrics=['ndcg_cut_5'])

,name,ndcg_cut_5
0,BR(BM25),0.0


In [16]:
indexer = pt.IterDictIndexer("/tmp/index", overwrite=True, stemmer='StanfordLemmatizer')
index_ref = indexer.index(documents)
index = pt.IndexFactory.of(index_ref)

bm25 = pt.BatchRetrieve(index, wmodel="BM25")

pt.Experiment([bm25], topics, qrels, eval_metrics=['ndcg_cut_5'])

,name,ndcg_cut_5
0,BR(BM25),0.693426
